## Corneal Endothelial Cell (CEC) Image Segmentation with U-Net Architecture

In [ ]:
!pip install imutils

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os, cv2, random, imutils, re
from matplotlib import pyplot as plt

from tensorflow.keras.layers import Dropout, Input, Dense, UpSampling2D, concatenate, Add, MaxPooling2D
from tensorflow.keras.layers import Layer, Dense, Flatten, Conv2D, Conv2DTranspose, Dropout
from tensorflow.keras.layers import add, BatchNormalization, ReLU, Activation, LeakyReLU
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import Sequential, Model
from tensorflow.keras.optimizers import Adam
import tensorflow_addons as tfa
from keras import backend as K

In [ ]:
def number(filename): 
    if 'cor' in filename: return int(filename[4:-4])
    elif 'cell' in filename: return int(filename[5:-4])
    elif 'guttae' in filename: return int(filename[7:-4])
    else: print('Error was found.') # if the label is missing.
        
def read_cec_patches(folder): # read images sequentially.
    images = []; names = [] # vectors to store the outputs.
    orderly_list = sorted(os.listdir(folder), key = number)
    for filename in orderly_list: # review each image as read, input.
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
        if img is not None: # if the image is not empty, proceed.
            images.append(img); names.append(filename)
    return images, names # returned vectors.

In [ ]:
path_synth = '../input/patches-bd/backup-synth'
images, fn_cor = read_cec_patches(path_synth + '/images')
cells, fn_cell = read_cec_patches(path_synth + '/masks/cells')
guttaes, fn_guttae = read_cec_patches(path_synth + '/masks/guttae')
images = np.array(images); images = np.expand_dims(images, axis = -1)
cells = np.array(cells); cells = np.expand_dims(cells, axis = -1) 
guttaes = np.array(guttaes); guttaes = np.expand_dims(guttaes, axis = -1)
backgrounds = cells + guttaes; backgrounds = 1 - backgrounds # contours.
masks = np.concatenate((backgrounds, cells, guttaes), axis = -1)

In [ ]:
print('CEC Images :::', images.shape); print('CEC Masks :::', masks.shape)

In [ ]:
def encoder_pixels(msk):
    labels = []
    cll = np.expand_dims(msk[..., 1], axis = 3)
    gtt = np.expand_dims(msk[..., 2], axis = 3)
    for n in range(len(msk)):
        c = cll[n] / 255
        g = gtt[n] / 255
        g[g == 1] = 2
        new_m = c + g
        labels.append(new_m)
    return np.asarray(labels)

labels = encoder_pixels(masks)
print('CEC Masks :::', labels.shape)
print('Pixels Class ::: ', np.unique(labels))

In [ ]:
def display_patches(artifact_list):
    plt.figure(figsize = (15, 15))
    graph = tf.keras.preprocessing.image.array_to_img
    title = ['CEC Image','Contour Mask','Cell Mask','Guttae Mask','Unified Mask']
    for x in range(len(artifact_list)):    
        plt.subplot(1,len(artifact_list),x+1); plt.title(title[x])
        plt.imshow(graph(artifact_list[x])); plt.axis('off')
    plt.show()

In [ ]:
e = random.sample(range(0, len(images)), 1)
cec_image = np.squeeze(images[e], axis = 0)
cec_mask = np.squeeze(masks[e], axis = 0)
contour = np.expand_dims(cec_mask[..., 0], axis = 2)
cell = np.expand_dims(cec_mask[..., 1], axis = 2)
guttae = np.expand_dims(cec_mask[..., 2], axis = 2)
display_patches([cec_image,contour,cell,guttae,labels[e[0]]])

In [ ]:
def counter_guttaes(guttae):
    c_guttae = []; rating = []; type_p = []
    for n in range(len(guttae)):
        cnts = cv2.findContours(guttae[n].copy(), cv2.RETR_EXTERNAL,
            cv2.CHAIN_APPROX_SIMPLE); cnts = imutils.grab_contours(cnts)
        c_guttae.append(len(cnts)) 
        rating.append('Yes' if c_guttae[n] > 0 else 'No')
        type_p.append('Healthy' if rating[n] == 'No' else 'Disease')
    cec_dict = {'Guttaes?':rating, 'Amount':c_guttae, 'Type Patch':type_p}
    cec_df = pd.DataFrame(cec_dict,columns=['Amount','Guttaes?','Type Patch'])

    return cec_df

In [ ]:
data_cec = counter_guttaes(guttaes); print(data_cec)

### Univariate Analysis

In [ ]:
# analizar la distribución de guttas a lo largo del dataset.
data_cec[{'Amount': ['min', 'max', 'median', 'skew']}].describe()

In [ ]:
import seaborn as sns
print('\tDistribution: Healthy VS Diseased Cells\n')
sns.countplot(x = data_cec['Type Patch']); plt.show()

### Model Building

In [ ]:
from sklearn.model_selection import train_test_split
train_images, tt_images, train_masks, tt_masks = train_test_split(images, labels, test_size = 0.2, random_state = 0)
test_images, val_images, test_masks, val_masks = train_test_split(tt_images, tt_masks, test_size = 0.4, random_state = 0)
print('Train Images :::', train_images.shape); print('Train Masks :::', train_masks.shape)
print('Test Images :::', test_images.shape); print('Test Masks :::', test_masks.shape)
print('Val Images :::', val_images.shape); print('Val Masks :::', val_masks.shape)
print("Classes CEC Patches ::: ", np.unique(train_masks))

In [ ]:
from tensorflow.keras.utils import normalize
train_images = normalize(train_images, axis = 1)
test_images = normalize(test_images, axis = 1)
val_images = normalize(val_images, axis = 1)

In [ ]:
from tensorflow.keras.utils import to_categorical
classes = 3; tr_cat = to_categorical(train_masks, num_classes = classes)
train_cat = tr_cat.reshape((train_masks.shape[0], train_masks.shape[1],
                            train_masks.shape[2], classes))

ts_cat = to_categorical(test_masks, num_classes = classes)
test_cat = ts_cat.reshape((test_masks.shape[0], test_masks.shape[1], 
                           test_masks.shape[2], classes))

vl_cat = to_categorical(val_masks, num_classes = classes)
val_cat = vl_cat.reshape((val_masks.shape[0], val_masks.shape[1], 
                           val_masks.shape[2], classes))

In [ ]:
def multi_unet_model(n_classes = 3, img_height = 96, img_width = 96, img_channels = 1, neurons = 32):
    inputs = Input((img_height, img_width, img_channels)); s = inputs
    # Contraction path
    c1 = Conv2D(neurons, (3, 3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(neurons, (3, 3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(neurons * 2, (3, 3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(neurons * 2, (3, 3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
     
    c3 = Conv2D(neurons * 4, (3, 3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(neurons * 4, (3, 3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
     
    c4 = Conv2D(neurons * 8, (3, 3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(neurons * 8, (3, 3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(c4)
    p4 = MaxPooling2D((2, 2))(c4)
     
    c5 = Conv2D(neurons * 16, (3, 3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(neurons * 16, (3, 3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(c5)
    
    # Expansion path 
    u6 = Conv2DTranspose(128, (2, 2), strides = (2, 2), padding = 'same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(neurons * 8, (3, 3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(neurons * 8, (3, 3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(c6)
     
    u7 = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(neurons * 4, (3, 3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(neurons * 4, (3, 3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(c7)
     
    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding = 'same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(neurons * 2, (3, 3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(neurons * 2, (3, 3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(c8)
     
    u9 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(c8)
    u9 = concatenate([u9, c1], axis = 3)
    c9 = Conv2D(neurons, (3, 3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(neurons, (3, 3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(c9)
     
    outputs = Conv2D(n_classes, (1, 1), activation = 'softmax')(c9)
     
    model = Model(inputs = [inputs], outputs = [outputs])

    model.summary()
    
    return model

In [ ]:
img_height = train_images.shape[1]
img_width  = train_images.shape[2]
img_channels = train_images.shape[3]
model = multi_unet_model(n_classes = classes, img_height = img_height,
        img_width = img_width, img_channels = img_channels, neurons = 32)

In [ ]:
opt = 'adam'; lost = 'categorical_crossentropy'; metric = 'accuracy'
model.compile(optimizer = opt, loss = [lost], metrics = [metric])

In [ ]:
def show_predictions(img, gt, pred):
    plt.figure(figsize = (16, 12)) # size fig.
    plt.subplot(231); plt.title('CEC Image')
    plt.imshow(img[:,:,0], cmap = 'gray')
    plt.subplot(232); plt.title('CEC Mask')
    plt.imshow(gt[:,:,0], cmap = 'inferno')
    plt.subplot(233); plt.title('Prediction')
    plt.imshow(pred, cmap = 'inferno'); plt.show()
    
def display_results(img_ls, gt_ls, pred_ls):
    fig, axs = plt.subplots(3, len(img_ls), figsize = (23,10))
    for x in range(len(img_ls)):    
        axs[0,x].imshow(img_ls[x][:,:,0], cmap = 'gray'); axs[0,x].axis('off')
        axs[1,x].imshow(gt_ls[x][:,:,0], cmap = 'inferno');axs[1,x].axis('off')
        axs[2,x].imshow(pred_ls[x], cmap = 'inferno'); axs[2,x].axis('off')
    plt.show()
    
def segment_patches(images = None, masks = None, samples = 1):
    if samples != 1:
        image_ls = []; mask_ls = []; predict_ls = []
        randomize = random.sample(range(0,len(images)),samples)
        for x in randomize:
            image = images[x]; mask = masks[x]
            image_norm = image[:,:,0][:,:,None]
            image_in = np.expand_dims(image_norm,0)
            pred_img = (model.predict(image_in))
            predict = np.argmax(pred_img,axis=3)[0,:,:]
            image_ls.append(image); mask_ls.append(mask)
            predict_ls.append(predict) 
        display_results(image_ls, mask_ls, predict_ls)
    else:
        img_norm = test_images[0][:,:,0][:,:,None]
        img_input = np.expand_dims(img_norm, 0)
        predicted_mask = (model.predict(img_input))
        predict_mask = np.argmax(predicted_mask,axis=3)[0,:,:]
        show_predictions(test_images[0], test_masks[0], predict_mask)

In [ ]:
from IPython.display import clear_output 
class display_callback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = None):
        clear_output(wait = True); segment_patches()
        print ('\nPrediction after epoch {}\n'.format(epoch+1))

In [ ]:
history = model.fit(train_images, train_cat, batch_size = 16, 
                    verbose =  1, epochs = 100, shuffle = False,
                    validation_data = (test_images, test_cat),
                    callbacks = [display_callback()])

In [ ]:
# plot the training and validation accuracy and loss at each epoch
fig = plt.figure(figsize = (20, 10))
plt.subplot(1, 2, 1)
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'b', label = 'Training loss')
plt.plot(epochs, val_loss, 'r', label = 'Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs'); plt.ylabel('Loss'); plt.legend()

plt.subplot(1, 2, 2)
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(epochs, acc, 'b', label = 'Training Accuracy')
plt.plot(epochs, val_acc, 'r', label = 'Validation Accuracy')
plt.title('Training and validation Accuracy')
plt.xlabel('Epochs'); plt.ylabel('Accuracy')
plt.legend(); plt.show()

In [ ]:
model.save('synth-trainer-100e.hdf5'); path = '../input/'
# model.load_weights(path + 'synth-trainer-50e.hdf5')

In [ ]:
_, acc_ts = model.evaluate(test_images, test_cat)
print("Accuracy Test is = ", (acc_ts * 100.0), "%")

In [ ]:
_, acc_vl = model.evaluate(val_images, val_cat)
print("Accuracy Val is = ", (acc_vl * 100.0), "%")

In [ ]:
y_pred = model.predict(test_images)
y_pred_argmax = np.argmax(y_pred, axis = 3)

In [ ]:
from keras.metrics import MeanIoU
IOU_keras = MeanIoU(num_classes = classes)  
IOU_keras.update_state(test_masks[:,:,:,0], y_pred_argmax)
print("Mean IoU =", IOU_keras.result().numpy())

In [ ]:
values = np.array(IOU_keras.get_weights()).reshape(classes, classes) 
contours_IoU = values[0,0] / (values[0,0] + values[0,1] + values[0,2] + values[1,0] + values[2,0])
cells_IoU = values[1,1] / (values[1,1] + values[1,0] + values[1,2] + values[0,1] + values[2,1])
guttaes_IoU = values[2,2] / (values[2,2] + values[2,0] + values[2,1] + values[0,2] + values[1,2])

In [ ]:
print("Intersection over Union (IoU) for contours is ::: ", contours_IoU) 
print("Intersection over Union (IoU) for cells is ::: ", cells_IoU) 
print("Intersection over Union (IoU) for guttaes is ::: ", guttaes_IoU) 

In [ ]:
segment_patches(test_images, test_masks, 7)

In [ ]:
segment_patches(val_images, val_masks, 7)

### Test: Real CEC Patches (Only For Train With Images From GAN).

In [ ]:
path_real = '../input/patches-bd/backup-real'
r_images, r_fn_cor = read_cec_patches(path_real + '/images')
r_cells, r_fn_cell = read_cec_patches(path_real + '/masks/cell')
r_guttaes, r_fn_guttae = read_cec_patches(path_real + '/masks/guttae')
r_images = np.array(r_images); r_images = np.expand_dims(r_images, axis = -1)
r_cells = np.array(r_cells); r_cells = np.expand_dims(r_cells, axis = -1) 
r_guttaes = np.array(r_guttaes); r_guttaes = np.expand_dims(r_guttaes, axis = -1)
r_backgrounds = r_cells + r_guttaes; r_backgrounds = 1 - r_backgrounds # contours.
r_masks = np.concatenate((r_backgrounds, r_cells, r_guttaes), axis = -1)
r_labels = encoder_pixels(r_masks) # encode pixels of the outgoing mask.
p_images = normalize(r_images, axis = 1); p_masks = r_labels 

In [ ]:
r_cat = to_categorical(p_masks, num_classes = classes)
p_cat = r_cat.reshape((p_masks.shape[0], p_masks.shape[1], 
                       p_masks.shape[2], classes))

_, r_acc = model.evaluate(p_images, p_cat)
print("Accuracy is = ", (r_acc * 100.0), "%")

In [ ]:
r_y_pred = model.predict(p_images)
r_y_pred_argmax = np.argmax(r_y_pred, axis = 3)

In [ ]:
from keras.metrics import MeanIoU
r_IOU_keras = MeanIoU(num_classes = classes)  
r_IOU_keras.update_state(p_masks[:,:,:,0], r_y_pred_argmax)
print("Mean IoU =", r_IOU_keras.result().numpy())

In [ ]:
r_values = np.array(r_IOU_keras.get_weights()).reshape(classes, classes) 
r_contours_IoU = r_values[0,0] / (r_values[0,0] + r_values[0,1] + r_values[0,2] + r_values[1,0] + r_values[2,0])
r_cells_IoU = r_values[1,1] / (r_values[1,1] + r_values[1,0] + r_values[1,2] + r_values[0,1] + r_values[2,1])
r_guttaes_IoU = values[2,2] / (r_values[2,2] + r_values[2,0] + r_values[2,1] + r_values[0,2] + r_values[1,2])

In [ ]:
print("Intersection over Union (IoU) for contours is ::: ", r_contours_IoU) 
print("Intersection over Union (IoU) for cells is ::: ", r_cells_IoU) 
print("Intersection over Union (IoU) for guttaes is ::: ", r_guttaes_IoU) 

In [ ]:
segment_patches(p_images, p_masks, 7)

### Predict on Large Image (CEC Image Not Seen)

In [ ]:
!pip install patchify

In [ ]:
from patchify import patchify, unpatchify
large_image = cv2.imread('../input/cec-images/01_20190108_132928_rc_a_POSSO JORGE.png', 0)
large_image = large_image[32:416, 16:208] # cut section of cec image for trainer.
patches = patchify(large_image, (96, 96), step = 96)  #Step=256 for 256 patches means no overlap

predicted_patches = []
for i in range(patches.shape[0]):
    for j in range(patches.shape[1]):
        single_patch = patches[i,j,:,:]       
        single_patch_norm = np.expand_dims(normalize(np.array(single_patch), axis=1),2)
        single_patch_input=np.expand_dims(single_patch_norm, 0)
        single_patch_prediction = (model.predict(single_patch_input))
        single_patch_predicted_img=np.argmax(single_patch_prediction, axis=3)[0,:,:]
        predicted_patches.append(single_patch_predicted_img)

predicted_patches = np.array(predicted_patches)
predicted_patches_reshaped = np.reshape(predicted_patches, (patches.shape[0], patches.shape[1], 96, 96) )
reconstructed_image = unpatchify(predicted_patches_reshaped, large_image.shape)
plt.hist(reconstructed_image.flatten()) 

plt.figure(figsize = (8, 8))
plt.subplot(221)
plt.title('Large Image')
plt.imshow(large_image, cmap = 'gray')
plt.subplot(222)
plt.title('Prediction of large Image')
plt.imshow(reconstructed_image, cmap = 'inferno')
plt.show()